# d-q座標系におけるLCL回路のインピーダンス
下に示すLCL回路のd-q座標上でのインピーダンスを求める。

<img src="LCL.png" style="zoom:40%;"/>

In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

# 並列演算記号'//'のオーバーライド
## 関数parallelを定義
def parallel(self, other):
    return self * other / (self + other)

## Symbol, Add, Mulのアトリビュート__floordiv__()をparallel()でオーバーライド
sp.Symbol.__floordiv__ = parallel
sp.Add.__floordiv__ = parallel
sp.Mul.__floordiv__ = parallel

# 回路要素のシンボル
R1, L1, R2, L2, C, Rp = sp.symbols('R_1 L_1 R_2 L_2 C R_p', real = True, positive = True)

# ラプラス演算子のシンボル（便宜上，real = Trueとする）
s = sp.symbols('s', real = True)

# 角周波数のシンボル
omega, omega1 = sp.symbols('omega omega_1', real = True)

In [2]:
# 静止(α-β)座標系でのインピーダンス
Zs = R1 + s * L1 + (R2 + s * L2) // (1 / (s * C) // Rp)

# ただのRL直列回路の場合
# Zs = R1 + s * L1

In [ ]:
Zs.ratsimp().collect(s)

In [ ]:
# 回転(d-q)座標系でのインピーダンス
Z = Zs.subs(s, (s - sp.I * omega1))

In [ ]:
# 対角成分
Zd = sp.re(Z).simplify()

In [ ]:
## 対角成分の式の整理
### 分子の抽出
Zd_num = sp.numer(Zd).expand().collect(s)

In [ ]:
Zd_num

In [ ]:
### 分子の係数
A5 = Zd_num.coeff(s, 5)
A4 = Zd_num.coeff(s, 4)
A3 = Zd_num.coeff(s, 3)
A2 = Zd_num.coeff(s, 2)
A1 = Zd_num.coeff(s, 1)
A0 = Zd_num.coeff(s, 0)

In [ ]:
A5

In [ ]:
A4

In [ ]:
A3

In [ ]:
A2

In [ ]:
A1

In [ ]:
A0

In [ ]:
### 分母の抽出
Zd_den = sp.denom(Zd).expand().collect(s)

In [ ]:
Zd_den

In [ ]:
### 分母の係数
B3 = Zd_den.coeff(s, 3)
B2 = Zd_den.coeff(s, 2)
B1 = Zd_den.coeff(s, 1)
B0 = Zd_den.coeff(s, 0)

In [ ]:
B3

In [ ]:
B2

In [ ]:
B1

In [ ]:
B0

In [ ]:
# 非対角成分（干渉項）
Zq = sp.im(Z).simplify()

In [ ]:
## 非対角成分の式の整理
### 分子の抽出
Zq_num = sp.numer(Zq).expand().collect(s)

In [ ]:
Zq_num

In [ ]:
### 分子の係数
C4 = Zq_num.coeff(s, 4)
C3 = Zq_num.coeff(s, 3)
C2 = Zq_num.coeff(s, 2)
C1 = Zq_num.coeff(s, 1)
C0 = Zq_num.coeff(s, 0)

In [ ]:
C4

In [ ]:
C3

In [ ]:
C2

In [ ]:
C1

In [ ]:
C0

In [ ]:
### 分母の抽出
Zq_den= sp.denom(Zq).expand().collect(s)

In [ ]:
Zq_den

In [ ]:
### 分母の係数
D4 = Zq_den.coeff(s, 4)
D3 = Zq_den.coeff(s, 3)
D2 = Zq_den.coeff(s, 2)
D1 = Zq_den.coeff(s, 1)
D0 = Zq_den.coeff(s, 0)

In [ ]:
D4

In [ ]:
D3

In [ ]:
D2

In [ ]:
D1

In [ ]:
D0

In [ ]:
# 関数の定義と回路定数の代入
# 回路定数
_R1 = 50e-3 # [Ω]
_L1 = 0.8e-3 # [H]
_R2 = 50e-3 # [Ω]
_L2 = 0.8e-3 # [H]
_C = 5e-6 # [F]
_Rp = 1e3 # [Ω]
_omega1 = 2 * sp.pi * 50 # [rad/s] 

# 対角成分関数の定義(lamnbdify)
_Zd = Zd.subs([(R1, _R1), (L1, _L1), (R2, _R2), (L2, _L2), (C, _C), (Rp, _Rp), (omega1, _omega1), (s, sp.I * omega)])
Zd_FRA = sp.lambdify(omega, _Zd, "numpy")

# 非対角成分（干渉項）関数の定義(lamnbdify)
_Zq = Zq.subs([(R1, _R1), (L1, _L1), (R2, _R2), (L2, _L2), (C, _C), (Rp, _Rp), (omega1, _omega1), (s, sp.I * omega)])
Zq_FRA = sp.lambdify(omega, _Zq, "numpy")

In [ ]:
_Zd

In [ ]:
_Zq

In [ ]:
# ボード線図のプロット（正相成分）
## データ生成
omega = np.logspace(1, 5, 200)
Zd_bode = Zd_FRA(omega)
Zq_bode = Zq_FRA(omega)

## ボード線図のプロット
fig, ax = plt.subplots(2, 1, figsize = (12, 8))
fig.patch.set_facecolor('lavender')

## 絶対値
ax[0].set_title(f'Positive-sequence impedance of an LCL circuit on the d-q reference frame')
ax[0].loglog(omega, np.abs(Zd_bode), ls = '-', label = r'$|Z_d(s)|$')
ax[0].loglog(omega, np.abs(Zq_bode), ls = '-', label = r'$|Z_q(s)|$')
ax[0].set_ylabel(r'Impedamce magnitude [$\Omega$]')
# ax[0].set_xlim(1e2, 1e4)
ax[0].set_ylim(0.01, 1e3)
ax[0].legend()
ax[0].grid()

## 偏角
ax[1].semilogx(omega, np.rad2deg(np.unwrap(np.angle(Zd_bode))), ls = '-', label = r'$\arg Z_d(s)$')
ax[1].semilogx(omega, np.rad2deg(np.unwrap(np.angle(Zq_bode))), ls = '-', label = r'$\arg Z_d(s)$')
# ax[1].set_xlim(1e2, 1e4)
ax[1].set_xlabel('Angular frequency [rad/s]')
ax[1].set_ylabel('Impedance angle [deg]')
ax[1].set_ylim(-360, 360)
ax[1].set_yticks(range(-360, 450, 90))
ax[1].legend()
ax[1].grid()

In [ ]:
# ボード線図のプロット（逆相成分）
## データ生成
omega = np.logspace(1, 5, 200)
Zd_bode = Zd_FRA(-omega)
Zq_bode = Zq_FRA(-omega)

## ボード線図のプロット
fig, ax = plt.subplots(2, 1, figsize = (12, 8))
fig.patch.set_facecolor('lavender')

## 絶対値
ax[0].set_title(f'Positive-sequence impedance of an LCL circuit on the d-q reference frame')
ax[0].loglog(omega, np.abs(Zd_bode), ls = '-', label = r'$|Z_d(s)|$')
ax[0].loglog(omega, np.abs(Zq_bode), ls = '-', label = r'$|Z_q(s)|$')
ax[0].set_ylabel(r'Impedamce magnitude [$\Omega$]')
# ax[0].set_xlim(1e2, 1e4)
ax[0].set_ylim(0.01, 1e3)
ax[0].legend()
ax[0].grid()

## 偏角
ax[1].semilogx(omega, np.rad2deg(np.unwrap(np.angle(Zd_bode))), ls = '-', label = r'$\arg Z_d(s)$')
ax[1].semilogx(omega, np.rad2deg(np.unwrap(np.angle(Zq_bode))), ls = '-', label = r'$\arg Z_d(s)$')
# ax[1].set_xlim(1e2, 1e4)
ax[1].set_xlabel('Angular frequency [rad/s]')
ax[1].set_ylabel('Impedance angle [deg]')
ax[1].set_ylim(-360, 360)
ax[1].set_yticks(range(-360, 450, 90))
ax[1].legend()
ax[1].grid()